#All the Necessary Modules

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
from keras import models
from keras.models import Model
from keras.layers import Input, Conv2D, MaxPool2D
from keras.layers.merge import concatenate
from keras.utils.vis_utils import plot_model as plot
from keras.layers.pooling import AveragePooling2D
from keras import layers
import tensorflow as tf
from keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt

input_shape = Input(shape= (299, 299, 3))
classes = 2
channel_axis = 3
path = ''

# Base Convolutional Part
With Batch Normalization and Rectified Linear Unit (linear activation function)

In [3]:
def conv2d_bn_relu(x, filters, num_row, num_col, padding = 'same', strides = (1, 1)):

  x = layers.Conv2D(filters, (num_row, num_col), strides = strides, padding = padding, use_bias = False)(x) # Here num_col and num_row are heights and weights of the column  
  x = layers.BatchNormalization(axis = 3, scale = False)(x)
  x = layers.Activation('relu')(x)
  return x

#First Inception Block

In [4]:
def inception_block_a(x):
  branch1x1 = conv2d_bn_relu(x, 64, 1, 1) # 64 filters of 1x1

  branch5x5 = conv2d_bn_relu(x, 48, 1, 1) # 48 filters of 1x1
  branch5x5 = conv2d_bn_relu(branch5x5, 64, 5, 5) # 64 filters of 5x5

  branch3x3_devidedpart = conv2d_bn_relu(x, 64, 1, 1) # 64 filters of 1x1
  branch3x3_devidedpart = conv2d_bn_relu(branch3x3_devidedpart, 96, 3, 3) # 96 filters of 3x3
  branch3x3_devidedpart = conv2d_bn_relu(branch3x3_devidedpart, 96, 3, 3) # 96 filters of 3x3

  branch_pool = layers.AveragePooling2D((3, 3), strides = (1, 1), padding = 'same')(x)
  branch_pool = conv2d_bn_relu(branch_pool, 32, 1, 1)
  x = layers.concatenate([branch1x1, branch5x5, branch3x3_devidedpart, branch_pool], axis = channel_axis, name = 'Fst_InceptionBlock1')
  return x

In [5]:
def inception_block_a1(x):
  branch1x1 = conv2d_bn_relu(x, 64, 1, 1) # 64 filters of 1x1

  branch5x5 = conv2d_bn_relu(x, 48, 1, 1) # 48 filters of 1x1
  branch5x5 = conv2d_bn_relu(branch5x5, 64, 5, 5) # 64 filters of 5x5

  branch3x3_devidedpart = conv2d_bn_relu(x, 64, 1, 1) # 64 filters of 1x1
  branch3x3_devidedpart = conv2d_bn_relu(branch3x3_devidedpart, 96, 3, 3) # 96 filters of 3x3
  branch3x3_devidedpart = conv2d_bn_relu(branch3x3_devidedpart, 96, 3, 3) # 96 filters of 3x3

  branch_pool = layers.AveragePooling2D((3, 3), strides = (1, 1), padding = 'same')(x)
  branch_pool = conv2d_bn_relu(branch_pool, 64, 1, 1)
  x = layers.concatenate([branch1x1, branch5x5, branch3x3_devidedpart, branch_pool], axis = channel_axis, name = 'Fst_InceptionBlock2')
  return x

In [6]:
def inception_block_a2(x):
  branch1x1 = conv2d_bn_relu(x, 64, 1, 1) # 64 filters of 1x1

  branch5x5 = conv2d_bn_relu(x, 48, 1, 1) # 48 filters of 1x1
  branch5x5 = conv2d_bn_relu(branch5x5, 64, 5, 5) # 64 filters of 5x5

  branch3x3_devidedpart = conv2d_bn_relu(x, 64, 1, 1) # 64 filters of 1x1
  branch3x3_devidedpart = conv2d_bn_relu(branch3x3_devidedpart, 96, 3, 3) # 96 filters of 3x3
  branch3x3_devidedpart = conv2d_bn_relu(branch3x3_devidedpart, 96, 3, 3) # 96 filters of 3x3

  branch_pool = layers.AveragePooling2D((3, 3), strides = (1, 1), padding = 'same')(x)
  branch_pool = conv2d_bn_relu(branch_pool, 64, 1, 1)
  x = layers.concatenate([branch1x1, branch5x5, branch3x3_devidedpart, branch_pool], axis = channel_axis, name = 'Fst_InceptionBlock3')
  return x

#First Reduction Block

In [7]:
def reduction_block_a(x):
  branch3x3 = conv2d_bn_relu(x, 384, 3, 3, strides = (2, 2), padding = 'valid')

  branch3x3_devidedpart = conv2d_bn_relu(x, 64, 1, 1) # 64 filters of 1x1
  branch3x3_devidedpart = conv2d_bn_relu(branch3x3_devidedpart, 96, 3, 3) # 96 filters of 3x3
  branch3x3_devidedpart = conv2d_bn_relu(branch3x3_devidedpart, 96, 3, 3, strides = (2, 2), padding = 'valid') # 96 filters of 3x3

  branch_pool = layers.MaxPooling2D((3, 3), strides = (2, 2))(x)
  x = layers.concatenate([branch3x3, branch3x3_devidedpart, branch_pool], axis = channel_axis, name = 'fst_ReductionBlock')
  return x

#Second Inception Block

In [8]:
def inception_block_b(x): # 17 x 17 x 768
  branch1x1 = conv2d_bn_relu(x, 192, 1, 1)

  branch7x7_devidedpart1 = conv2d_bn_relu(x, 128, 1, 1)
  branch7x7_devidedpart1 = conv2d_bn_relu(branch7x7_devidedpart1, 128, 1, 7)
  branch7x7_devidedpart1 = conv2d_bn_relu(branch7x7_devidedpart1, 192, 7, 1)

  branch7x7_devidedpart2 = conv2d_bn_relu(x, 128, 1, 1)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 128, 7, 1)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 128, 1, 7)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 128, 7, 1)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 192, 1, 7)


  branch_pool = layers.AveragePooling2D((3, 3), strides = (1, 1), padding = 'same')(x)
  branch_pool = conv2d_bn_relu(branch_pool, 192, 1, 1)
  x = layers.concatenate([branch1x1, branch7x7_devidedpart1, branch7x7_devidedpart2, branch_pool], axis = channel_axis, name = 'sec_InceptionBlock1')
  return x

In [9]:
def inception_block_b1(x): # 17 x 17 x 768
  branch1x1 = conv2d_bn_relu(x, 192, 1, 1)

  branch7x7_devidedpart1 = conv2d_bn_relu(x, 160, 1, 1)
  branch7x7_devidedpart1 = conv2d_bn_relu(branch7x7_devidedpart1, 160, 1, 7)
  branch7x7_devidedpart1 = conv2d_bn_relu(branch7x7_devidedpart1, 192, 7, 1)

  branch7x7_devidedpart2 = conv2d_bn_relu(x, 160, 1, 1)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 160, 7, 1)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 160, 1, 7)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 160, 7, 1)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 192, 1, 7)


  branch_pool = layers.AveragePooling2D((3, 3), strides = (1, 1), padding = 'same')(x)
  branch_pool = conv2d_bn_relu(branch_pool, 192, 1, 1)
  x = layers.concatenate([branch1x1, branch7x7_devidedpart1, branch7x7_devidedpart2, branch_pool], axis = channel_axis, name = 'sec_InceptionBlock2')
  return x

In [10]:
def inception_block_b2(x): # 17 x 17 x 768
  branch1x1 = conv2d_bn_relu(x, 192, 1, 1)

  branch7x7_devidedpart1 = conv2d_bn_relu(x, 160, 1, 1)
  branch7x7_devidedpart1 = conv2d_bn_relu(branch7x7_devidedpart1, 160, 1, 7)
  branch7x7_devidedpart1 = conv2d_bn_relu(branch7x7_devidedpart1, 192, 7, 1)

  branch7x7_devidedpart2 = conv2d_bn_relu(x, 160, 1, 1)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 160, 7, 1)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 160, 1, 7)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 160, 7, 1)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 192, 1, 7)


  branch_pool = layers.AveragePooling2D((3, 3), strides = (1, 1), padding = 'same')(x)
  branch_pool = conv2d_bn_relu(branch_pool, 192, 1, 1)
  x = layers.concatenate([branch1x1, branch7x7_devidedpart1, branch7x7_devidedpart2, branch_pool], axis = channel_axis, name = 'sec_InceptionBlock3')
  return x

In [11]:
def inception_block_b3(x): # 17 x 17 x 768
  branch1x1 = conv2d_bn_relu(x, 192, 1, 1)

  branch7x7_devidedpart1 = conv2d_bn_relu(x, 192, 1, 1)
  branch7x7_devidedpart1 = conv2d_bn_relu(branch7x7_devidedpart1, 192, 1, 7)
  branch7x7_devidedpart1 = conv2d_bn_relu(branch7x7_devidedpart1, 192, 7, 1)

  branch7x7_devidedpart2 = conv2d_bn_relu(x, 192, 1, 1)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 192, 7, 1)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 192, 1, 7)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 192, 7, 1)
  branch7x7_devidedpart2 = conv2d_bn_relu(branch7x7_devidedpart2, 192, 1, 7)


  branch_pool = layers.AveragePooling2D((3, 3), strides = (1, 1), padding = 'same')(x)
  branch_pool = conv2d_bn_relu(branch_pool, 192, 1, 1)
  x = layers.concatenate([branch1x1, branch7x7_devidedpart1, branch7x7_devidedpart2, branch_pool], axis = channel_axis, name = 'sec_InceptionBlock4')
  return x

#Second Reduction Block

In [12]:
def reduction_block_b(x): # 8 x 8 x 1280
  branch3x3 = conv2d_bn_relu(x, 192, 1, 1)
  branch3x3 = conv2d_bn_relu(branch3x3, 320, 3, 3, strides= (2 , 2), padding = 'valid')

  branch7x7x3_devidedpart = conv2d_bn_relu(x, 192, 1, 1)
  branch7x7x3_devidedpart = conv2d_bn_relu(branch7x7x3_devidedpart, 192, 1, 7) 
  branch7x7x3_devidedpart = conv2d_bn_relu(branch7x7x3_devidedpart, 192, 7, 1)
  branch7x7x3_devidedpart = conv2d_bn_relu(branch7x7x3_devidedpart, 192, 3, 3, strides = (2, 2), padding = 'valid') 

  branch_pool = layers.MaxPooling2D((3, 3), strides = (2, 2))(x)
  x = layers.concatenate([branch3x3, branch7x7x3_devidedpart, branch_pool], axis = channel_axis, name = 'sec_reductionBlock')
  return x

#Third Inception Block

In [13]:
def inception_block_c(x):
    branch1x1 = conv2d_bn_relu(x, 320, 1, 1)

    branch3x3 = conv2d_bn_relu(x, 384, 1, 1)
    branch3x3_1 = conv2d_bn_relu(branch3x3, 384, 1, 3)
    branch3x3_2 = conv2d_bn_relu(branch3x3, 384, 3, 1)
    branch3x3Final = layers.concatenate([branch3x3_1, branch3x3_2], axis = channel_axis)

    branch3x3_devidedpart = conv2d_bn_relu(x, 448, 1, 1)
    branch3x3_devidedpart = conv2d_bn_relu(branch3x3_devidedpart, 384, 3, 3)
    branch3x3_devidedpart_1 = conv2d_bn_relu(branch3x3_devidedpart, 384, 1, 3)
    branch3x3_devidedpart_2 = conv2d_bn_relu(branch3x3_devidedpart, 384, 3, 1)
    branch3x3_devidedpartFinal = layers.concatenate([branch3x3_devidedpart_1, branch3x3_devidedpart_2], axis = channel_axis, name= 'Third_BlockMiddlePart')



    branch_pool = layers.AveragePooling2D((3, 3), strides = (1, 1), padding = 'same')(x)
    branch_pool = conv2d_bn_relu(branch_pool, 192, 1, 1)
    x = layers.concatenate([branch1x1, branch3x3Final, branch3x3_devidedpartFinal, branch_pool], axis = channel_axis, name = 'Third_InceptionBlock')
    return x

In [14]:
def inception_block_c1(x):
    branch1x1 = conv2d_bn_relu(x, 320, 1, 1)

    branch3x3 = conv2d_bn_relu(x, 384, 1, 1)
    branch3x3_1 = conv2d_bn_relu(branch3x3, 384, 1, 3)
    branch3x3_2 = conv2d_bn_relu(branch3x3, 384, 3, 1)
    branch3x3Final = layers.concatenate([branch3x3_1, branch3x3_2], axis = channel_axis)

    branch3x3_devidedpart = conv2d_bn_relu(x, 448, 1, 1)
    branch3x3_devidedpart = conv2d_bn_relu(branch3x3_devidedpart, 384, 3, 3)
    branch3x3_devidedpart_1 = conv2d_bn_relu(branch3x3_devidedpart, 384, 1, 3)
    branch3x3_devidedpart_2 = conv2d_bn_relu(branch3x3_devidedpart, 384, 3, 1)
    branch3x3_devidedpartFinal = layers.concatenate([branch3x3_devidedpart_1, branch3x3_devidedpart_2], axis = channel_axis, name= 'Third_BlockMiddlePart1')



    branch_pool = layers.AveragePooling2D((3, 3), strides = (1, 1), padding = 'same')(x)
    branch_pool = conv2d_bn_relu(branch_pool, 192, 1, 1)
    x = layers.concatenate([branch1x1, branch3x3Final, branch3x3_devidedpartFinal, branch_pool], axis = channel_axis, name = 'Third_InceptionBlock1')
    return x

# Main Model Assemble

In [15]:
# Starting Of the model and the 1st Step
# In this step 299 x 299 x 3 will be the input shape and  output shape will be 149 x 149 x 32

x = conv2d_bn_relu(input_shape, 32, 3, 3, strides = (2, 2), padding = 'valid') 
x = conv2d_bn_relu(x, 32, 3, 3, padding = 'valid') # 147 x 147 x 32 output shape
x = conv2d_bn_relu(x, 64, 3, 3) # 147 x 147 x 64 output shape
x = layers.MaxPooling2D((3, 3), strides = (2, 2))(x) # 73 x 73 x 64 output shape

# Second Step
x = conv2d_bn_relu(x, 80, 1, 1, padding = 'valid') # 73 x 73 x 80 output shape
x = conv2d_bn_relu(x, 192, 3, 3, padding = 'valid') # 73 x 73 x 192 output shape
x = layers.MaxPooling2D((3, 3), strides = (2, 2))(x) # 35 x 35 x 192 output shape

# Third Step and 1st Inception Block
# First Inception block35 x 35 x 256 where it will be for 3 times a x 3
x = inception_block_a(x)
x = inception_block_a1(x)
x = inception_block_a2(x)

x = reduction_block_a(x)

# Fourth Step and 2nd Inception Block
x = inception_block_b(x)
x = inception_block_b1(x)
x = inception_block_b2(x)
x = inception_block_b3(x)

x = reduction_block_b(x)

# Fifth Step and 3rd Inception Block
x = inception_block_c(x)
x = inception_block_c1(x)

# Last Layer before fully connected layer
x = layers.GlobalAveragePooling2D(name = 'avg_pool')(x)

# The fully connected layer
x = layers.Dense(classes, activation = 'softmax', name = 'predictions')(x)




# Creating the actual model and it's Summary
inputs = input_shape
model = models.Model(inputs, x, name = 'Muzzle_Inception') 
model.compile(optimizer=Adam(learning_rate=0.0001),loss = 'categorical_crossentropy', metrics= ['accuracy'])
model.summary()

Model: "Muzzle_Inception"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 299, 299, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 149, 149, 32  864         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 149, 149, 32  96         ['conv2d[0][0]']                 
 alization)                     )                                                  

In [16]:
train_path = '/content/drive/MyDrive/dogsVScats_Small_Scale/training'
valid_path = '/content/drive/MyDrive/dogsVScats_Small_Scale/validation'
test_path = '/content/drive/MyDrive/dogsVScats_Large_Scale/test'

In [17]:
train_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input) \
    .flow_from_directory(directory=train_path, target_size=(299,299), classes=['dogs', 'cats'], batch_size=10, shuffle = True)
valid_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input) \
    .flow_from_directory(directory=valid_path, target_size=(299,299), classes=['dogs', 'cats'], batch_size=10, shuffle = True)
test_batches = ImageDataGenerator(preprocessing_function=tf.keras.applications.inception_v3.preprocess_input) \
    .flow_from_directory(directory=test_path, target_size=(299,299), classes=['dogs', 'cats'], batch_size=10, shuffle=False)

Found 8005 images belonging to 2 classes.
Found 2023 images belonging to 2 classes.
Found 2498 images belonging to 2 classes.


In [ ]:
r = model.fit(x=train_batches, validation_data=valid_batches, epochs=5)

Epoch 1/5
620/801 [======================>.......] - ETA: 4:00 - loss: 0.6810 - accuracy: 0.6215

In [ ]:
plt.style.use("ggplot")
plt.plot(r.history['accuracy'])
plt.plot(r.history['val_accuracy'])
plt.title('Model Accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Training Acc', 'Validation Acc'], loc='lower right')
plt.show()
 
plt.plot(r.history['loss'])
plt.plot(r.history['val_loss'])
plt.title('Model Loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Training Loss', 'Validation Loss'], loc='upper right')
plt.show()